In [15]:
import sys
print(sys.path)
sys.path.append("../../KE/")
print(sys.path)

['/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-BacDive/kg-bacdive/notebook', '/Users/marcin/.pyenv/versions/3.10.11/lib/python310.zip', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10/lib-dynload', '', '/Users/marcin/Documents/KBase/KE/MGnify/MLhackathon/venv/lib/python3.10/site-packages']
['/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-BacDive/kg-bacdive/notebook', '/Users/marcin/.pyenv/versions/3.10.11/lib/python310.zip', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10/lib-dynload', '', '/Users/marcin/Documents/KBase/KE/MGnify/MLhackathon/venv/lib/python3.10/site-packages', '../../KE/']


In [38]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

from KE.classifier.prepare import split_dataset
from KE.dataclean import tables

In [8]:
data = pd.read_csv("../../try1/edges.tsv", header=0, sep="\t")

In [9]:
data

,subject,edge_label,object,relation,provided_by
0,NCBITaxon:1955243,biolink:growsIn-PLACEHOLDER,DSMZ:1669,NaN,BACDIVE:158156
1,NCBITaxon:1873457,biolink:growsIn-PLACEHOLDER,DSMZ:92,NaN,BACDIVE:139807
2,NCBITaxon:1848903,biolink:growsIn-PLACEHOLDER,DSMZ:1609,NaN,BACDIVE:132461
3,NCBITaxon:1978225,biolink:growsIn-PLACEHOLDER,DSMZ:830,NaN,BACDIVE:132310
4,NCBITaxon:1413221,biolink:growsIn-PLACEHOLDER,DSMZ:65,NaN,BACDIVE:130726
...,...,...,...,...,...
16586,NCBITaxon:83456,biolink:growsIn-PLACEHOLDER,DSMZ:9,NaN,BACDIVE:9819
16587,NCBITaxon:184914,biolink:growsIn-PLACEHOLDER,DSMZ:9,NaN,BACDIVE:9298
16588,NCBITaxon:184914,biolink:growsIn-PLACEHOLDER,DSMZ:9,NaN,BACDIVE:8728
16589,NCBITaxon:1410668,biolink:growsIn-PLACEHOLDER,DSMZ:339,NaN,BACDIVE:2872


In [12]:
data_pairs = data[['subject','object']].drop_duplicates()
data_pairs

,subject,object
0,NCBITaxon:1955243,DSMZ:1669
1,NCBITaxon:1873457,DSMZ:92
2,NCBITaxon:1848903,DSMZ:1609
3,NCBITaxon:1978225,DSMZ:830
4,NCBITaxon:1413221,DSMZ:65
...,...,...
16575,NCBITaxon:1054,DSMZ:28
16579,NCBITaxon:1121296,DSMZ:634a
16580,NCBITaxon:328814,DSMZ:1713
16585,NCBITaxon:66870,DSMZ:65


In [23]:
#data_pairs_clean = tables.rows_and_cols_quant_filter(data_pairs)
data_pairs_clean = data_pairs.groupby('object').filter(lambda x : len(x)>9)

In [24]:
data_pairs_clean

,subject,object
1,NCBITaxon:1873457,DSMZ:92
3,NCBITaxon:1978225,DSMZ:830
4,NCBITaxon:1413221,DSMZ:65
5,NCBITaxon:1122939,DSMZ:830
6,NCBITaxon:1931,DSMZ:65
...,...,...
16570,NCBITaxon:687207,DSMZ:514
16574,NCBITaxon:316274,DSMZ:67
16575,NCBITaxon:1054,DSMZ:28
16585,NCBITaxon:66870,DSMZ:65


In [32]:
# Splitting the data into features and target labels
X = data_pairs_clean[['subject']]
y = data_pairs_clean['object']

# Convert the input labels column to a categorical type if it isn't
X['subject'] = X['subject'].astype('category')

# Convert categorical columns to integers
X['subject'] = X['subject'].cat.codes

/var/folders/7w/ksxs106s7_sg14v6cb2y70vr0000gn/T/ipykernel_84407/29537296.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['subject'] = X['subject'].astype('category')
/var/folders/7w/ksxs106s7_sg14v6cb2y70vr0000gn/T/ipykernel_84407/29537296.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['subject'] = X['subject'].cat.codes


In [33]:
# Train the model
train_data = Pool(data=X, label=y, cat_features=[0])
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
model.fit(train_data)



0:	learn: 4.1240367	total: 322ms	remaining: 5m 22s
1:	learn: 3.9735757	total: 789ms	remaining: 6m 33s
2:	learn: 3.8709637	total: 1.25s	remaining: 6m 55s
3:	learn: 3.7979437	total: 1.64s	remaining: 6m 48s
4:	learn: 3.7391951	total: 2.1s	remaining: 6m 58s
5:	learn: 3.6927696	total: 2.57s	remaining: 7m 6s
6:	learn: 3.6556609	total: 3.03s	remaining: 7m 10s
7:	learn: 3.6247246	total: 3.27s	remaining: 6m 45s
8:	learn: 3.6004826	total: 3.5s	remaining: 6m 25s
9:	learn: 3.5794454	total: 3.97s	remaining: 6m 33s
10:	learn: 3.5615275	total: 4.45s	remaining: 6m 39s
11:	learn: 3.5457981	total: 4.92s	remaining: 6m 44s
12:	learn: 3.5322772	total: 5.39s	remaining: 6m 49s
13:	learn: 3.5218763	total: 5.55s	remaining: 6m 31s
14:	learn: 3.5121283	total: 6.03s	remaining: 6m 35s
15:	learn: 3.5027914	total: 6.51s	remaining: 6m 40s
16:	learn: 3.4954862	total: 6.98s	remaining: 6m 43s
17:	learn: 3.4884800	total: 7.46s	remaining: 6m 46s
18:	learn: 3.4826899	total: 7.93s	remaining: 6m 49s
19:	learn: 3.4772407	tota

In [ ]:
# Predict on new data
new_data = pd.DataFrame({'subject': ['A', 'C']})
new_data['subject'] = new_data['subject'].astype('category').cat.codes
predictions = model.predict(new_data)

print(predictions)

In [34]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train the model
train_data = Pool(data=X_train, label=y_train, cat_features=[0])
test_data = Pool(data=X_test, label=y_test, cat_features=[0])


In [36]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(train_data, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 4.1550605	total: 263ms	remaining: 4m 22s
100:	learn: 3.4074414	total: 35.8s	remaining: 5m 18s
200:	learn: 3.3361299	total: 1m 14s	remaining: 4m 54s
300:	learn: 3.3072929	total: 1m 52s	remaining: 4m 20s
400:	learn: 3.2893649	total: 2m 30s	remaining: 3m 44s
500:	learn: 3.2769949	total: 3m 8s	remaining: 3m 7s
600:	learn: 3.2685522	total: 3m 46s	remaining: 2m 30s
700:	learn: 3.2625302	total: 4m 24s	remaining: 1m 52s
800:	learn: 3.2581014	total: 5m 2s	remaining: 1m 15s
900:	learn: 3.2545324	total: 5m 40s	remaining: 37.5s
999:	learn: 3.2515567	total: 6m 18s	remaining: 0us


In [39]:
# Predict on test data
y_pred = model.predict(test_data)
y_pred_proba = model.predict_proba(test_data)[:,1]  # Probabilities for the positive class

# Print metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_proba))

Accuracy: 0.1955521472392638

Classification Report:
               precision    recall  f1-score   support

      DSMZ:1       0.45      0.22      0.30        76
    DSMZ:104       0.17      0.03      0.05        34
   DSMZ:1040       0.00      0.00      0.00         4
   DSMZ:104b       0.00      0.00      0.00        16
   DSMZ:104c       0.00      0.00      0.00         6
  DSMZ:1076b       0.00      0.00      0.00        11
     DSMZ:11       0.25      0.04      0.06        28
    DSMZ:110       0.00      0.00      0.00        13
   DSMZ:110a       0.00      0.00      0.00         5
   DSMZ:1113       0.00      0.00      0.00         3
   DSMZ:1122       0.00      0.00      0.00         4
   DSMZ:1166       0.00      0.00      0.00         4
    DSMZ:119       0.00      0.00      0.00         3
   DSMZ:1193       0.00      0.00      0.00         1
  DSMZ:1203a       0.00      0.00      0.00         8
    DSMZ:141       0.00      0.00      0.00         3
   DSMZ:141c       0.00    

/Users/marcin/Documents/KBase/KE/MGnify/MLhackathon/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marcin/Documents/KBase/KE/MGnify/MLhackathon/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marcin/Documents/KBase/KE/MGnify/MLhackathon/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

ValueError: multi_class must be in ('ovo', 'ovr')

In [25]:
df_train, df_val, df_test = split_dataset(data_pairs_clean, "object", "./", random_seed=42)

In [30]:
# train pool
X_train = df_train['subject']
y_train = df_train['object']
X_train_pool = Pool(
    data=X_train,
    label=y_train,
    # cat_features=np.where(X_train.dtypes != float)[0],
)
# val pool
X_val = df_val['subject']
y_val = df_val['object']
X_val_pool = Pool(
    data=X_val,
    label=y_val,
    # cat_features=np.where(X_val.dtypes != float)[0],
)

KeyError: 0